# Ejemplo de criptosistema ELGamal

### Participantes: 
- Nikita Stetskiy
- Manuel Hidalgo Carmona
- Pablo Pérez Méndez

Supongamos el siguiente ejemplo:
Bob desea enviar a Alice si está o no aprobada y desea que no lo sepa nadie más, para ello enviará cifrado el texto plano APROBADO a Alice.

1. Comenzamos eligiendo un número primo perfecto muy elevado, más grande en función del mensaje que queremos encriptar. Usamos `openssl prime -generate -safe -bits 64` para generar por ejemplo `16554795285782121983`
2. Elegimos un elemento g como Es preferible que g sea un elemento primitivo. Lo obtenemos como:
```
Kk = GF(p)
g = Kk.primitive_element()
```
3. Cada usuario elige aleatoriamente a tal que 0 < a < p − 1 ; le servirá como clave privada para el descifrado y generará una clave pública como terna de (p,g,y), donde y es `y = g^a mod p`

### Cifrado
Ciframos el mensaje m (0< m < p-1)
1. Se elige aleatoriamente un entero k tq: 2 ≤ k ≤ p-1.
2. Es enviada a Alice la pareja (y1, y2), donde y1 e y2 se calculan como:
```
y1 = g^k mod p
y2 = m*y^k mod p
```

### Descifrado
Para descifrar realizamos el siguiente cálculo: `y1^-a mod p` donde`y1^-a`representa el inverso de y1^a mod p necesario conocer la clave privada a.
1. y1^-a mod p = y1^(p-1-a) mod p. Lo expresamos como y1a = y1^(p-1-a) mod p
2. Resolvemos `y1a*y2 mod p` para desencriptar el mensaje. `des = y1a*y2 mod p`

## Ejemplo implementado

Primero declaramos 4 funciones que nos facilitarán la codificación y decodificación del mensaje,
donde nosotros hemos declarado un alfabeto de 27 caracteres de los cuales el primer caracter es basura, por lo que
podemos considerar que son 26.  
También definimos el primo que vamos a usar y el mensaje.

In [5]:
import random  

alphabet = "*ABCDEFGHIJKLMNOPQRSTUVWXYZ"
PRIMO = 16554795285782121983 # Primo aleatorio con openssl prime -generate -safe -bits 64
SECRETO = 'APROBADO'

#Trasformar letras a indices (letras a lista de numeros) (biyeccion f)
def f(secret):
    secretB10 = []
    for x in secret:
        secretB10.append(alphabet.index(x)+1)
    return secretB10

#Trasformar indices a letras (lista de numeros a letras)
def df(secret):
    secretB26 = ""
    for x in secret:
        secretB26 += alphabet[x-1]
    return secretB26

#Base 26 a base 10
def base10(l):
    res = 0
    i = len(l)
    while i > 0:
        res += l[i - 1] * (26)**(len(l) - i)
        i = i - 1
    return res

#Base 10 a base 26
def base26(l):
    res = []
    numel = l
    mod = 1
    while numel>0:
        mod = numel % (26)
        res.insert(0,mod)
        numel = numel//(26)
    return res

### Generamos la clave pública (p,g,y)

In [6]:
## Generación de clave pública
# Primo aleatorio seguro 
p = PRIMO

#Generador
Kk = GF(p)
g = Kk.primitive_element()

#Clave privada aleatoria
a = random.randint(1,p-1)#21702

y = mod(g**a, p)

print("p = "+ str(p)+ "\tPrimo aleatorio seguro " )
print("g = "+ str(g)+ "\tGenerador de GF(p)")
print("a = "+ str(a)+ "\tClave privada aleatoria")
print("y = "+ str(y))
print("Su clave pública (p,g,y)")

p = 16554795285782121983	Primo aleatorio seguro 
g = 5	Generador de GF(p)
a = 13735770723789946498	Clave privada aleatoria
y = 7361591867610993757
Su clave pública (p,g,y)


### Encriptación
1. Codificamos el mensaje en base 10
2. Generamos k para Bob,(Para crear una "clave pública")
3. Generamos y1 e y2 (mensaje cifrado) para enviarlos a Alice

In [7]:
## Encryption
#mensaje a encriptar
secret = SECRETO
print("\nMensaje a encriptar: " + secret)

#Codificacion a base 10
m = base10(f(secret))                               
print("m = "+str(m) + "\t Codificacion en base 10")

#clave del emisor Bob
k = random.randint(2,p-1)
print("k = "+ str(k) + "\t Clave Bob")

y1 = mod(g**k, p)
y2 = mod(m*(y**k), p)

print("\nBob Cifra el mensaje calculando y1 y y2")
print("y1 = "+str(y1))
print(df(base26(int(y1))))
print("y2 = "+str(y2))
print(df(base26(int(y2))))
print("\nEnvia y1 e y2 a Alice")


Mensaje a encriptar: APROBADO
m = 21548300530	 Codificacion en base 10
k = 12852643622952529311	 Clave Bob

Bob Cifra el mensaje calculando y1 y y2
y1 = 13972793928405672281
DOIWLSD*ELJKSX
y2 = 12138091748781527801
CVD*UHCGFWCHEX

Envia y1 e y2 a Alice


### Desencriptado
1. Generamos el inverso de y1^a mod p
2. Recuperamos el mensaje cifrado

In [8]:
## Decryption
y1_a = mod(y1**(p-1-a),p) 
decrypted = mod(y1_a*y2,p)

print("\nAlice desencripta el mensaje:")
print("decrypted = "+str(decrypted))
print(df(base26(int(decrypted))))


Alice desencripta el mensaje:
decrypted = 21548300530
APROBADO
